In [1]:
import fast_stats
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

In [2]:
pred = np.random.randint(0, 2, (10, 512, 512)).flatten()
actual = np.random.randint(0, 2, (10, 512, 512)).flatten()

In [6]:
def fast_binary_recall(pred, actual):
    x, y = fast_stats._fast_binary_recall(pred, actual)
    return x / y

In [7]:
%%timeit
_ = fast_binary_recall(actual, pred)

5.97 ms ± 246 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit
_ = precision_score(actual, pred)

In [ ]:
%%timeit
_ = recall_score(actual, pred)

In [ ]:
%%timeit
_ = f1_score(actual, pred)

In [ ]:
# don't actually need to flatten them for fast-stats
pred = np.random.randint(0, 2, (10, 512, 512), dtype=np.uint64)
actual = np.random.randint(0, 2, (10, 512, 512), dtype=np.uint64)

In [ ]:
%%timeit
_ = fast_stats.precision(actual, pred)

In [ ]:
%%timeit
_ = fast_stats.recall(actual, pred)

In [ ]:
%%timeit
_ = fast_stats.f1_score(actual, pred)

In [ ]:
assert np.allclose(
    fast_stats.precision(actual.flatten(), pred.flatten()),
    precision_score(actual.flatten(), pred.flatten())
)
assert np.allclose(
    fast_stats.recall(actual.flatten(), pred.flatten()),
    recall_score(actual.flatten(), pred.flatten())
)
assert np.allclose(
    fast_stats.f1_score(actual.flatten(), pred.flatten()),
    f1_score(actual.flatten(), pred.flatten())
)

In [ ]:
%%timeit
_ =np.logical_and(actual, pred).sum() / pred.sum()

In [ ]:
%%timeit
_ = fast_stats.binary_precision(actual, pred)